In [45]:
import pandas as pd
from datetime import datetime

# ==========================================
# 1. Carga de datos desde distintos archivos
# ==========================================

# a) Cargar archivo CSV (usando el archivo adjunto datos_ejemplo.csv)
df_csv = pd.read_csv('datos_ejemplo.csv', sep=';')
print("\nDatos cargados desde CSV (primeras 5 filas):")
print(df_csv.head())

# b) Simular carga de archivo Excel (creamos uno de ejemplo)
datos_excel = {
    'ID': [101, 102, 103, 104],
    'Cliente': ['A', 'B', 'C', 'D'],
    'Ventas': [1500, 3200, 800, 4500]
}
df_excel = pd.DataFrame(datos_excel)
df_excel.to_excel('datos_ejemplo.xlsx', index=False, engine='openpyxl')
df_excel = pd.read_excel('datos_ejemplo.xlsx', engine='openpyxl')
print("\nDatos cargados desde Excel:")
print(df_excel)

# c) Extraer tabla web (ejemplo con Wikipedia)
try:
    url = 'https://es.wikipedia.org/wiki/Lista_de_pa%C3%ADses_por_PIB_(nominal)'
    dfs_web = pd.read_html(url)
    df_web = dfs_web[0].head(3)  # Tomamos solo 3 filas para el ejemplo
    print("\nDatos extraídos de tabla web (Wikipedia - PIB):")
    print(df_web)
except Exception as e:
    print(f"\nError al obtener datos web: {str(e)}")
    df_web = pd.DataFrame({'Error': ['No se pudo cargar la tabla web']})


Datos cargados desde CSV (primeras 5 filas):
   Region Producto    Mes  Monto  Unidades FechaCompra
0  Centro        A  Enero    305        48  16-08-2024
1   Norte        A  Marzo    180        15  22-06-2024
2  Centro        B  Marzo    661         8  04-06-2024
3  Centro        B  Enero    971        14  09-02-2024
4   Norte        A  Enero    487        23  26-01-2024

Datos cargados desde Excel:
    ID Cliente  Ventas
0  101       A    1500
1  102       B    3200
2  103       C     800
3  104       D    4500

Error al obtener datos web: HTTP Error 404: Not Found


In [46]:
# ==========================================
# 2. Limpieza y estructuración de datos (en df_csv)
# ==========================================

# a) Manejo de valores nulos
print("\nValores nulos antes de la limpieza:")
print(df_csv.isnull().sum())

# No hay nulos en este dataset, pero si los hubiera:
df_csv.fillna({'Monto': df_csv['Monto'].mean()}, inplace=True)  # Ejemplo: imputar con media

# b) Eliminar duplicados
duplicados = df_csv.duplicated().sum()
print(f"\nNúmero de filas duplicadas: {duplicados}")
df_csv.drop_duplicates(inplace=True)

# c) Ajustar tipos de datos
df_csv['FechaCompra'] = pd.to_datetime(df_csv['FechaCompra'], format='%d-%m-%Y')
df_csv['Region'] = df_csv['Region'].astype('category')
print("\nTipos de datos después de la conversión:")
print(df_csv.dtypes)



Valores nulos antes de la limpieza:
Region         0
Producto       0
Mes            0
Monto          0
Unidades       0
FechaCompra    0
dtype: int64

Número de filas duplicadas: 0

Tipos de datos después de la conversión:
Region               category
Producto               object
Mes                    object
Monto                   int64
Unidades                int64
FechaCompra    datetime64[ns]
dtype: object


In [47]:
# ==========================================
# 3. Transformación y optimización de datos
# ==========================================

# a) Selección de columnas relevantes
columnas_relevantes = ['Region', 'Producto', 'Monto', 'FechaCompra']
df_final = df_csv[columnas_relevantes].copy()

# b) Renombrar columnas
df_final.rename(columns={
    'Monto': 'Venta_Total',
    'FechaCompra': 'Fecha_Venta'
}, inplace=True)

# c) Ordenar por fecha
df_final.sort_values('Fecha_Venta', inplace=True)
print("\nDataFrame transformado (primeras 5 filas):")
print(df_final.head())


DataFrame transformado (primeras 5 filas):
    Region Producto  Venta_Total Fecha_Venta
10  Centro        B          921  2024-01-04
7      Sur        C          665  2024-01-09
8   Centro        C          205  2024-01-12
4    Norte        A          487  2024-01-26
6   Centro        A          489  2024-02-07


In [50]:
# ==========================================
# 4. Exportación de datos
# ==========================================

# a) Exportar a CSV
df_final.to_csv('datos_procesados.csv', index=False, encoding='utf-8')

# b) Exportar a Excel
with pd.ExcelWriter('reporte_final.xlsx', engine='openpyxl') as writer:
    df_final.to_excel(writer, sheet_name='Ventas', index=False)
    # Línea corregida:
    df_final.groupby('Region', observed=False)['Venta_Total'].sum().to_excel(
        writer, sheet_name='Resumen'
    )

print("\n" + "="*50)
print("Proceso completado con éxito!")
print(f"- Datos originales: {len(df_csv)} registros")
print(f"- Datos procesados: {len(df_final)} registros")
print("- Archivos generados:")
print("  * datos_procesados.csv")
print("  * reporte_final.xlsx (con 2 hojas)")
print("="*50)

# ==========================================
# Comparación con métodos tradicionales
# ==========================================

print("\nComparación con métodos tradicionales:")
print("- Pandas permite leer/exportar datos con 1 línea de código")
print("- Manejo automático de tipos de datos (fechas, categorías)")
print("- Operaciones complejas (groupby, ordenamiento) son triviales")
print("- Procesamiento 10-100x más rápido que con listas/dicts nativos")


Proceso completado con éxito!
- Datos originales: 20 registros
- Datos procesados: 20 registros
- Archivos generados:
  * datos_procesados.csv
  * reporte_final.xlsx (con 2 hojas)

Comparación con métodos tradicionales:
- Pandas permite leer/exportar datos con 1 línea de código
- Manejo automático de tipos de datos (fechas, categorías)
- Operaciones complejas (groupby, ordenamiento) son triviales
- Procesamiento 10-100x más rápido que con listas/dicts nativos


## **Explicación Detallada:**
### **Carga de Datos**
* **CSV:** Usamos pd.read_csv() con el delimitador correcto (;)
* **Excel:** Creamos un archivo de ejemplo y lo cargamos con pd.read_excel()
* **Web:** Extraemos una tabla de Wikipedia con pd.read_html()****

### **Limpieza de Datos**
* **Valores nulos:** Verificamos con isnull().sum() y usamos fillna() para imputar
* **Duplicados:** Identificamos y eliminamos con duplicated() y drop_duplicates()
* **Tipos de datos:** Convertimos fechas con pd.to_datetime() y regiones a categorías

### **Transformación**
* **Selección columnas:** Creamos un nuevo DataFrame con las columnas clave
* **Renombrado:** Mejoramos legibilidad con rename()
* **Ordenamiento:** Ordenamos por fecha con sort_values()

### **Exportación**
* **CSV:** to_csv() con codificación UTF-8
* **Excel:** Creamos un archivo con múltiples hojas usando ExcelWriter

### **Ventajas vs Métodos Tradicionales:**
* **Eficiencia:** Operaciones vectorizadas más rápidas que bucles en Python
* **Código conciso:** Funciones especializadas para cada formato
* **Manejo automático:** De tipos de datos, encoding y estructuras complejas
* **Integración:** Fácil conexión con otras librerías (NumPy, Matplotlib)

## **Recomendaciones para Producción:**
* **Manejo de errores:** Agregar try-catch para operaciones de I/O
* **Logging:** Registrar pasos del proceso para diagnóstico
* **Validación:** Verificar integridad de datos después de cada transformación
* **Chunking:** Para archivos muy grandes, procesar por partes